### Q1 Running Elastic

❯ curl localhost:9200
{
  "name" : "637a035dca56",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "tk1goW4mSFC0wsqDrsjNZQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}

Answer 1: 42f05b9372a9a4a470db3b52817899b99a76ee73

### Getting the data

In [9]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2 Indexing the data

In [10]:
import os
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

# Creating a local client for the search
es_client = Elasticsearch('http://localhost:9200')

# Index setting
index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }

# Naming the index
index_name = "course-questions"

# Creating the index using the settings and the name
es_client.indices.create(index=index_name, body=index_settings)

# Using the index to iterate through all documents
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:07<00:00, 121.82it/s]


Answer 2: Index

### Q3 Searching

In [11]:
query = "How do I execute a command in a running docker container?"

def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    # Searhing using the query
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append({
            "_source": hit['_source'],
            "_score": hit['_score']
        })

    return result_docs

elastic_search(query)

[{'_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  '_score': 84.050095},
 {'_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\

Answer 3: 84.05

### Q4 Filtering

In [12]:
def elastic_search_filtered(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    # Searhing using the query
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

elastic_search_filtered(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

Answer 4: How do I copy files from a different folder into docker container’s working directory?

### Q5 Building a prompt

In [30]:
def build_prompt(query, search_results):
    context_entries = []
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    for record in search_results:
        # print("Printing a record: ", record)
        formatted_entry = context_template.format(question=record['question'], text=record['text'])
        #print("Printing the formatted entry: ", formatted_entry)
        context_entries.append(formatted_entry)

    context = "\n\n".join(context_entries)

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

final_prompt = build_prompt(query, elastic_search_filtered(query))

print(final_prompt)
len(final_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

1462

Answer 5: 1462

### Q6 Tokens

In [31]:
import sys
sys.path.append('/home/dmitry/Documents/Programming/Python/llm-zoomcamp/.venv/lib/python3.12/site-packages/')
import tiktoken
# Using tokenization algorithm for gpt-4o: https://community.openai.com/t/whats-the-new-tokenization-algorithm-for-gpt-4o/746708
# encoding = tiktoken.get_encoding("o200k_base")
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(final_prompt)
len(tokens)

322

Answer 6: 322

### Bonus: generating the answer

In [32]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

response = llm(final_prompt)
print(response)

To execute a command in a running Docker container, follow these steps:

1. First, find the container ID of the running container by using the following command:
   ```sh
   docker ps
   ```

2. Once you have the container ID, you can execute a command in that specific container. For example, to start a bash session inside the container, use the following command:
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container.


### Bonus: calculating the costs

Cost for gpt-4o:

    Input: $0.005 / 1K tokens
Output: $0.015 / 1K tokens



In [35]:
# Prompt cost
p_cost = len(tokens) * 0.005 / 1000
print("The prompt is", len(final_prompt), "characters long")
print("Cost of our final prompt is $", p_cost)

# Answer cost
a_tokens = encoding.encode(response)
a_cost = len(a_tokens) * 0.015 / 1000
print("The response is", len(response), "characters long")
print("Cost of the response is $", a_cost)

The prompt is 1462 characters long
Cost of our final prompt is $ 0.00161
The response is 493 characters long
Cost of the response is $ 0.001665
